# 上传 233 张 CXR 到 Hugging Face Dataset

**在 Kaggle 中运行**，需添加：
1. 数据集：`simhadrisadaram/mimic-cxr-dataset`
2. 数据集：`xiruizhu1111/clean-data`（含 CSV）
3. Secrets：`zhuxirui11` = 你的 HF Write token

In [ ]:
import os
import csv
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, create_repo

HF_TOKEN = UserSecretsClient().get_secret("zhuxirui11")
if not HF_TOKEN:
    raise SystemExit("请在 Kaggle Add-ons → Secrets 添加 zhuxirui11 = 你的 HF token")

os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
# 查找 CSV 和图片路径
CSV_CANDIDATES = [
    "/kaggle/input/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/datasets/xiruizhu1111/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/clean-train-data/mimic_eval_single_image_final_233.csv",
]
csv_path = None
for p in CSV_CANDIDATES:
    if os.path.exists(p):
        csv_path = p
        break
if not csv_path:
    raise FileNotFoundError("未找到 CSV，请添加 clean-data 数据集")
print(f"✅ CSV: {csv_path}")

In [ ]:
def find_image(raw_path):
    raw = raw_path.strip()
    if os.path.exists(raw):
        return raw
    for prefix in [
        "/kaggle/input/mimic-cxr-dataset",
        "/kaggle/input/datasets/simhadrisadaram/mimic-cxr-dataset",
    ]:
        if "mimic-cxr-dataset" in raw:
            rel = raw.split("mimic-cxr-dataset/", 1)[-1]
        else:
            rel = raw.lstrip("/")
        full = os.path.join(prefix, rel)
        if os.path.exists(full):
            return full
    return None

In [ ]:
api = HfApi(token=HF_TOKEN)
username = api.whoami()["name"]
repo_id = f"{username}/cxr-233-images"
create_repo(repo_id, repo_type="dataset", private=False, exist_ok=True, token=HF_TOKEN)
print(f"✅ Dataset: https://huggingface.co/datasets/{repo_id}")

In [ ]:
rows = []
with open(csv_path, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    fieldnames = list(reader.fieldnames)
    for i, row in enumerate(reader):
        raw_path = row.get("Image_Path", "")
        local_path = find_image(raw_path)
        if local_path:
            fname = f"{row['subject_id']}_{i}.jpg"
            api.upload_file(
                path_or_fileobj=local_path,
                path_in_repo=fname,
                repo_id=repo_id,
                repo_type="dataset",
                token=HF_TOKEN,
            )
            url = f"https://huggingface.co/datasets/{repo_id}/resolve/main/{fname}"
            row["Image_Path"] = url
            print(f"  [{i+1}/233] {fname}")
        else:
            row["Image_Path"] = ""
            print(f"  [{i+1}/233] ⚠️ 未找到")
        rows.append(row)

In [ ]:
out_path = "/kaggle/working/reports-data-with-urls.csv"
with open(out_path, "w", encoding="utf-8", newline="") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames, extrasaction="ignore")
    w.writeheader()
    w.writerows(rows)
print(f"\n✅ 已生成: {out_path}")
print("👉 在 Kaggle Output 中下载，复制到 medgamma-frontend/lib/reports-data.csv")

In [ ]:
# 生成下载链接（运行后点击即可下载）
from IPython.display import FileLink, display
display(FileLink("reports-data-with-urls.csv", result_html_prefix="📥 点击下载: "))